In [1]:
import sqlite3
import numpy as np
import pandas as pd
import io
import zipfile
import re
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from tqdm import tqdm
from nltk.tokenize import word_tokenize
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yousu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yousu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
conn = sqlite3.connect('eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

[('zipfiles',)]
num
name
content


In [3]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [4]:
def extract_content(row):
    binary_data = row['content']
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            subtitle_content = zip_file.read(zip_file.namelist()[0])
            return subtitle_content.decode('latin-1')  # Assuming the content is latin-1 encoded text

# Apply the function to each row of the DataFrame
df['extracted_content'] = df.apply(extract_content, axis=1)

In [5]:
df

,num,name,content,extracted_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
...,...,...,...,...
82493,9521935,the.prophets.game.(2000).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
82494,9521937,west.beirut.(1998).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
82495,9521938,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati..."
82496,9521940,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis..."


In [6]:
df.drop_duplicates(subset=['name'], inplace=True)

In [7]:
df

,num,name,content,extracted_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
...,...,...,...,...
82490,9521931,immanence.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xba\x96\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nSupport ..."
82493,9521935,the.prophets.game.(2000).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
82494,9521937,west.beirut.(1998).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
82495,9521938,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati..."


In [8]:
df = df.iloc[0:2000]

In [9]:
def remove_pattern(content):
    pattern = r'(\r\n)|(ï»¿)'
    return re.sub(pattern, " ", content)
%time df['extracted_content'] = df['extracted_content'].apply(remove_pattern)

CPU times: total: 1.66 s
Wall time: 4.2 s


In [10]:
def remove_series(text):
    return re.sub(r"\d+\s+\d{2}:\d{2}:\d{2},\d{3}\s+-->\s+\d{2}:\d{2}:\d{2},\d{3}\s+", "", text)

%time df['extracted_content'] = df['extracted_content'].apply(remove_series)

CPU times: total: 734 ms
Wall time: 1.49 s


In [11]:
def clean_subtitle(subtitle):
    # Remove the specified series
    subtitle = re.sub(r"Dialogue:\s+0,0:00:06.00,0:00:12.074.*\n", "", subtitle)
    # Remove the specified series
    subtitle = re.sub(r"\{.*?\}|\\N|\\[a-z]+\d*|{\*.*?}|{\d+\}|\[[^\]]+\]|\([^\)]+\)", "", subtitle)
    # Remove lines starting with [Script Info], [V4+ Styles], and [Events]
    subtitle = re.sub(r"^\[(Script Info|V4\+ Styles|Events)\].*\n", "", subtitle, flags=re.MULTILINE)
    # Remove lines starting with Format:, Style:, and Dialogue:
    subtitle = re.sub(r"^(Format|Style|Dialogue):\s.*\n", "", subtitle, flags=re.MULTILINE)
    # Remove leading and trailing whitespace
    subtitle = subtitle.strip()
    return subtitle

In [12]:
%time df['extracted_content'] = df['extracted_content'].apply(clean_subtitle)

CPU times: total: 328 ms
Wall time: 846 ms


In [13]:
def clean_subtitle(subtitle):
    # Remove the specified series
    subtitle = re.sub(r"Dialogue:\s+0,0:00:06.00,0:00:12.074.*\n", "", subtitle)
    # Remove lines starting with [Script Info], [V4+ Styles], and [Events]
    subtitle = re.sub(r"^\[(Script Info|V4\+ Styles|Events)\].*\n", "", subtitle, flags=re.MULTILINE)
    # Remove lines starting with Format:, Style:, and Dialogue:
    subtitle = re.sub(r"^(Format|Style|Dialogue):\s.*\n", "", subtitle, flags=re.MULTILINE)
    # Remove leading and trailing whitespace
    subtitle = subtitle.strip()
    return subtitle

In [14]:
%time df['extracted_content'] = df['extracted_content'].apply(clean_subtitle)

CPU times: total: 328 ms
Wall time: 686 ms


In [16]:
def clean_subtitle(subtitle):
    # Remove the specified series
    subtitle = re.sub(r"\{.*?\}|\\N|\\[a-z]+\d*|{\*.*?}|{\d+\}|\[[^\]]+\]|\([^\)]+\)", "", subtitle)
    # Remove lines starting with [Script Info], [V4+ Styles], and [Events]
    subtitle = re.sub(r"^\[(Script Info|V4\+ Styles|Events)\].*\n", "", subtitle, flags=re.MULTILINE)
    # Remove lines starting with Format:, Style:, and Dialogue:
    subtitle = re.sub(r"^(Format|Style|Dialogue):\s.*\n", "", subtitle, flags=re.MULTILINE)
    # Remove leading and trailing whitespace
    subtitle = subtitle.strip()
    return subtitle

In [17]:
%time df['extracted_content'] = df['extracted_content'].apply(clean_subtitle)

CPU times: total: 328 ms
Wall time: 713 ms


In [18]:
lemmatizer = WordNetLemmatizer()
def preprocess(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    text=re.sub("[^A-Za-z0-9./:']"," ",text)
    # Convert to lowercase
    text = text.lower()
    

    return text

In [19]:
%time df['extracted_content'] = df['extracted_content'].apply(preprocess)

CPU times: total: 2.81 s
Wall time: 5.4 s


In [20]:
def clean_subtitle(subtitle):
    # Remove the specified series
    subtitle = re.sub(r"\{.*?\}|\\N|\\[a-z]+\d*|{\*.*?}|{\d+\}|\[[^\]]+\]|\([^\)]+\)", "", subtitle)
    # Remove lines starting with [Script Info], [V4+ Styles], and [Events]
    subtitle = re.sub(r"^\[(script info|v4\+ styles|events)\].*\n", "", subtitle, flags=re.MULTILINE)
    # Remove lines starting with Format:, Style:, and Dialogue:
    subtitle = re.sub(r"^(format|style|dialogue):\s.*\n", "", subtitle, flags=re.MULTILINE)
    # Remove leading and trailing whitespace
    subtitle = subtitle.strip()
    return subtitle

In [21]:
%time df['extracted_content'] = df['extracted_content'].apply(clean_subtitle)

CPU times: total: 359 ms
Wall time: 770 ms


In [22]:
df['name'] = df['name'].str.replace('.', ' ')
df['name'] = df['name'].str.replace('.eng.1cd', '')

In [24]:
df['name'] = df['name'].str.replace(' eng 1cd', '')

In [25]:
df

,num,name,content,extracted_content
0,9180533,the message (1976),b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,watch any video online with open subtitles fre...
1,9180583,here comes the grump s01 e09 joltin jack in bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,ah there's princess dawn and terry with the ...
2,9180592,yumis cells s02 e13 episode 2 13 (2022),b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,yumi's cells 2 episode 36 extremely polite yum...
3,9180594,yumis cells s02 e14 episode 2 14 (2022),b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,watch any video online with open subtitles fre...
4,9180600,broker (2022),b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,watch any video online with open subtitles fre...
...,...,...,...,...
3093,9195008,the roads not taken (2020),b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x11\x8c\x...,watch any video online with open subtitles fre...
3094,9195009,all or nothing arsenal s01 e03 stay in the gam...,b'PK\x03\x04\x14\x00\x00\x00\x08\x003\x8c\x99V...,get close guys come on. friday night guys. ...
3095,9195010,sonata (2021),b'PK\x03\x04\x14\x00\x00\x00\x08\x00%\x8c\x99V...,watch any video online with open subtitles fre...
3096,9195011,yan zhi kou (1987),b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1d\x8c\x...,advertise your product or brand here contact w...


In [26]:
def remove_name(text):
    text = re.sub(r'episode\s\d+\s\d+','',text)
    return text

%time df['name'] = df['name'].apply(remove_name)

CPU times: total: 0 ns
Wall time: 1.71 ms


In [27]:
def remove_double_spaces(content):
    return re.sub(r'\s+', ' ', content)

# Apply the function to each row of the DataFrame
df['extracted_content'] = df['extracted_content'].apply(remove_double_spaces)

In [28]:
df['extracted_content'] = df['extracted_content'].apply(lambda x: x.strip())

In [29]:
from nltk.tokenize import word_tokenize
CHUNK_SIZE = 500  # Adjust this size to control the number of words in each chunk
OVERLAP_SIZE = 100  # Adjust this size to control the overlap between chunks

def chunk_content(content):
    # Tokenize the input content into words
    words = word_tokenize(content)
    
    # Initialize a list to hold the chunks
    chunks = []
    
    # Create chunks based on the specified chunk size and overlap size
    for i in range(0, len(words), CHUNK_SIZE - OVERLAP_SIZE):
        # Calculate the end index of the current chunk
        end_index = i + CHUNK_SIZE
        
        # Create a chunk by slicing the words list
        chunk = words[i:end_index]
        
        # Convert the chunk list back to a single string
        chunk_text = ' '.join(chunk)
        
        # Add the chunk text to the chunks list
        chunks.append(chunk_text)
    
    # Return the list of chunks
    return chunks

In [30]:
%time df['chunk_content'] = df['extracted_content'].apply(chunk_content)

CPU times: total: 35.2 s
Wall time: 1min 2s


In [32]:
import sqlite3
import json

# Function to persist data to SQLite database
def persist_to_database(database_path, data):
    conn = sqlite3.connect(database_path)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS records
                 (num TEXT, name TEXT, embeddings TEXT)''')
    
    for record in data:
        num = record['num']
        name = record['name']
        embeddings = json.dumps(record['embeddings'])  # Convert embeddings list to JSON string
        c.execute("INSERT INTO records (num, name, embeddings) VALUES (?, ?, ?)", (num, name, embeddings))
    
    conn.commit()
    conn.close()

# Initialize BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Example DataFrame with columns: num, name, clean_content
df1 = df.head(100)

# Chunk size and overlap
chunk_size = 512  # Adjust this according to your needs
overlap_size = 100  # Adjust this according to your needs

# Example data processing, chunking, and encoding
data_to_persist = []
for index, row in df1.iterrows():
    num = row['num']
    name = row['name']
    clean_content = row['extracted_content']
    chunks = [clean_content[i:i+chunk_size] for i in range(0, len(clean_content), chunk_size - overlap_size)]
    for i, chunk in enumerate(chunks):
        embeddings = model.encode([chunk])[0].tolist()
        data_to_persist.append({'num': f"{num}_chunk{i+1}", 'name': name, 'embeddings': embeddings})

# Database path
database_path = 'C:\\Users\\yousu\\OneDrive\\Python_Files\\Search Engine\\chroma1.sqlite3'

# Persist data to SQLite database
persist_to_database(database_path, data_to_persist)

# Function to retrieve top 10 similar records based on query
def get_top_10_unique_names(query, database_path):
    conn = sqlite3.connect(database_path)
    c = conn.cursor()
    
    query_embedding = model.encode([query])
    similarities = []
    c.execute("SELECT * FROM records")
    for row in c.fetchall():
        record_num, record_name, record_embeddings = row
        embeddings = json.loads(record_embeddings)
        embeddings = np.array(embeddings).reshape(1, -1)
        similarity = cosine_similarity(query_embedding, embeddings)[0][0]
        similarities.append((record_name, similarity))
    
    # Sort by similarity score and extract unique names
    sorted_names = [name for name, _ in sorted(similarities, key=lambda x: x[1], reverse=True)]
    unique_names = []
    for name in sorted_names:
        if name not in unique_names:
            unique_names.append(name)
            if len(unique_names) == 10:
                break
    
    conn.close()
    return unique_names

# Example usage
query = "from muhammad the messenger of god to heraclius the emperor of byzantium. greeting to him who is the follower of righteous guidance. i bid you to hear the divine call. i am the messenger of god to the people accept islam for your salvation."
top_10_unique_names = get_top_10_unique_names(query, database_path)
print(top_10_unique_names)

['the message (1976)', 'the myth (2005)', 'doctor lawyer s01 e14  (2022)', 'the governor s01 e01  (1995)', 'reetur s01 e04 the second time will be easier (2020)', 'westworld s04 e05 zhuangzi (2022)', 'avaete semente da vinganca (1985)', 'the anarchists s01 e03 currency (2022)', 'mo jong yuen so hatyi (1992)', 'yumis cells s02 e13  (2022)']
